In [10]:
%load_ext autoreload
%autoreload 2

import os
from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt
import madmom

import sys
sys.path.append('../src')
from preprocessing import get_dataset
from models import OLSPatchRegressor

na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [3]:
music_dir  = '../data/music_speech/music_wav/'
speech_dir = '../data/music_speech/speech_wav/'

max_samples = -1

X, Y = get_dataset(music_dir, speech_dir, num_samples=max_samples, hpool=0, wpool=0)
print('Train Set Shape')
print(X.shape, Y.shape)

Y = (Y + 1) / 2 

Train Set Shape
(128, 74, 1500, 1) (128,)


# Train CNN using KERAS

In [4]:
batch_size = 8
num_classes = 2
epochs = 109

# --------------------------------------------
num_frequencies = X.shape[1]
num_timesteps   = X.shape[2]
num_channels    = X.shape[3]
input_shape = num_frequencies, num_timesteps, num_channels

# DEFINE MODEL
model = Sequential()
model.add(MaxPooling2D(pool_size=(1, 3), input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(num_frequencies, 3),
                 activation='relu'))

model.add(Conv2D(1, kernel_size=(1, 1), activation='sigmoid'))
model.add(Lambda(lambda x: K.mean(x, axis=[1,2])))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [5]:
t_ind = 96
XTrain = X[:t_ind]
YTrain = Y[:t_ind]
XTest  = X[t_ind:]
YTest  = Y[t_ind:]

print(XTrain.shape, YTrain.shape)
print(XTest.shape, YTest.shape)

(96, 74, 1500, 1) (96,)
(32, 74, 1500, 1) (32,)


In [6]:
model.fit(XTrain, YTrain,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(XTest, YTest))
score = model.evaluate(XTest, YTest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 96 samples, validate on 32 samples
Epoch 1/109
96/96 [==============================] - 0s 5ms/step - loss: 0.6877 - acc: 0.5000 - val_loss: 0.6804 - val_acc: 0.4688
Epoch 2/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6855 - acc: 0.5208 - val_loss: 0.6719 - val_acc: 0.5625
Epoch 3/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6739 - acc: 0.4896 - val_loss: 0.6867 - val_acc: 0.4375
Epoch 4/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6549 - acc: 0.5208 - val_loss: 0.6535 - val_acc: 0.7812
Epoch 5/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6564 - acc: 0.5938 - val_loss: 0.6448 - val_acc: 0.6250
Epoch 6/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6437 - acc: 0.5625 - val_loss: 0.6269 - val_acc: 0.8750
Epoch 7/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6272 - acc: 0.6042 - val_loss: 0.6214 - val_acc: 0.7188
Epoch 8/109
96/96 [====================

In [14]:
model_path = '../models/keras/'
os.makedirs(model_path, exist_ok=True)

model.save(os.path.join(model_path, 'cnn_on_spectros_{:2.2f}.h5'.format(score[1])))